In [1]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ING Lion's den 2024!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#### Michal Bryzik(*)
#### Kacper Gruca($)
#### Michal Niegierewicz(&)
#### Jan Slusarek(^)

# Zadanie 1 

# Jesli robicie to na VScode ogarnijcie jadro!!!!!

"""""
Plan dzialania: (Jbc to kazdy podaje tu swoje propozycje, jak sie kto z czym np. niezgadza niech napisze w nawiasie)
1. Wczytac dane i szybko je opisac liczebnosc nulle itd. 
2. Ja bym tutaj posortowal zmienne w macierzy tzn. kolumna 1-16 numeryczne a od 17-do konca binarne
3. Wlasciwe czyszczenie danych, elminacja, aproksymacja, asocjacja itd.
4. Statystyki opisowe, wykresy (rozklady, heatmapy, boxploty itd. )
5. Transformacja danych MinMax, standaryzacja itd. (ogolnie manipulacja tak zeby w razie czego logit byl jak najbardziej dojebany)
6. Trenowanie Logita
7. Wybor najlepszego i sprawdzenie z zalozeniami 




"""""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv('../data/input/development_sample.csv') # Wczytujemy^
data.head() # Pierwsze 5 rekordow^

,ID,customer_id,application_date,target,Application_status,Var1,Var2,Var3,Var4,Var5,...,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,_r_
0,11034977,32537148,01Feb2010 0:00:00,0.0,Approved,1,2.0,1,7800,99,...,1,1,0,6768.42,0.00,0,0,0,3899,0.267045
1,11034978,32761663,01Feb2010 0:00:00,0.0,Approved,1,1.0,2,11100,78,...,0,0,15000,5937.66,3870.60,0,0,0,3899,0.533040
2,11034979,32701063,01Feb2010 0:00:00,0.0,Approved,2,3.0,1,2400,15,...,2,2,0,5647.77,1463.08,0,0,10,3899,0.861555
3,11034980,32386786,01Feb2010 0:00:00,0.0,Approved,3,1.0,2,11800,30,...,0,0,0,3594.90,21563.78,0,0,10,3899,0.506451
4,11034981,32692110,02Feb2010 0:00:00,NaN,Rejected,1,2.0,1,10200,72,...,2,2,0,9908.45,0.00,0,0,20,3899,0.210913


In [3]:
data.shape # Wymiar macierzy danych treningowych^

(50000, 36)

In [4]:
data.isnull().sum() # Zliczamy liczbe nulli dla kazdej kolumny^

ID                        0
customer_id               0
application_date          0
target                13282
Application_status        0
Var1                      0
Var2                   1403
Var3                   1403
Var4                      0
Var5                      0
Var6                      0
Var7                      0
Var8                  29173
Var9                      0
Var10                 37538
Var11                     0
Var12                 37538
Var13                     0
Var14                     0
Var15                     0
Var16                     0
Var17                    40
Var18                 37416
Var19                 29173
Var20                     0
Var21                     0
Var22                     0
Var23                     0
Var24                     0
Var25                 10088
Var26                 19883
Var27                     0
Var28                     0
Var29                     0
Var30                     0
_r_                 

In [5]:
data.info() # Analityka z typami danych dla kazdej kolumny ^ 
# Hmmm pasuje je posortowac kolumnami tzn. najpierw numeryczne potem kategoryczne porzadkowe i na koncu binarne^ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  50000 non-null  int64  
 1   customer_id         50000 non-null  int64  
 2   application_date    50000 non-null  object 
 3   target              36718 non-null  float64
 4   Application_status  50000 non-null  object 
 5   Var1                50000 non-null  int64  
 6   Var2                48597 non-null  float64
 7   Var3                48597 non-null  object 
 8   Var4                50000 non-null  int64  
 9   Var5                50000 non-null  int64  
 10  Var6                50000 non-null  int64  
 11  Var7                50000 non-null  float64
 12  Var8                20827 non-null  float64
 13  Var9                50000 non-null  int64  
 14  Var10               12462 non-null  float64
 15  Var11               50000 non-null  int64  
 16  Var1

### Sort columns by datetypes

In [6]:
# Define columns
date_columns = ['application_date', 'Var13'] 
numeric_columns = ['ID', 'customer_id', '_r_', 'Var9', 'Var10']  
categorical_columns = []
binary_columns = ['target', 'Application_status'] 

In [7]:
# Modify date columns
data_h=data.copy()

data_h['application_date'] = pd.to_datetime(data_h['application_date'], errors='coerce')# Zmiana tych dat na wlasiwy typ
data_h['Var13'] = pd.to_datetime(data_h['Var13'], format='%d%b%Y', errors='coerce') # Var13 te daty co maja nieregularne nazwy

In [8]:
# Sprawdzenie jaka cala resztta ma typ danych 
for column in data_h.columns:
    if column in date_columns + numeric_columns + binary_columns:
        continue  
    elif column in ['Var1', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var15', 'Var16', 'Var17', 'Var20', 'Var21', 'Var22', 'Var23', 'Var24', 'Var25', 'Var26', 'Var29', 'Var30']:
        numeric_columns.append(column)
    elif column in ['Var2', 'Var3', 'Var11', 'Var12', 'Var14']:
        categorical_columns.append(column)
    elif column in ['Var18', 'Var19', 'Var27', 'Var28']:
        binary_columns.append(column)

# kolejnosc sortowania wg ktorych ulozone sa kolumny 
sorted_columns = date_columns + numeric_columns + categorical_columns + binary_columns

# wybór wartosci posortowanych wg. zdefiniowanej przez nas kategorii 
sorted_data = data_h[sorted_columns]

In [9]:
sorted_data.head()

# Wychodzi na to ze po transformacji tych dat niektore rekordy dla Var13 zostały puste 
# Trzeba to przeanalizowac

,application_date,Var13,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,2010-02-01,2004-11-26,11034977,32537148,0.267045,5640,NaN,1,7800,99,...,1,5,NaN,2,0.0,Approved,1.0,NaN,0,0
1,2010-02-01,1996-09-16,11034978,32761663,0.533040,3000,NaN,1,11100,78,...,2,2,NaN,4,0.0,Approved,NaN,1.0,0,0
2,2010-02-01,1989-05-06,11034979,32701063,0.861555,6240,4300.0,2,2400,15,...,1,5,5.0,1,0.0,Approved,NaN,NaN,0,0
3,2010-02-01,1987-12-06,11034980,32386786,0.506451,4560,2500.0,3,11800,30,...,2,6,5.0,1,0.0,Approved,NaN,0.0,0,0
4,2010-02-02,1989-05-18,11034981,32692110,0.210913,5160,NaN,1,10200,72,...,1,5,NaN,0,NaN,Rejected,1.0,NaN,0,0


In [10]:
data_h['Var13'].isnull().sum(0)

784

In [11]:
sorted_data['application_date'].isnull().sum(0)

0

In [12]:
# Tworzymy kopię DataFrame 'data', aby zachować oryginalne dane
data_copy = data.copy()

# Konwertujemy 'Var13' na typ daty w kopii, zapisując wynik w nowej kolumnie
data_copy['Var13_converted'] = pd.to_datetime(data_copy['Var13'], format='%d%b%Y', errors='coerce')

# Znajdujemy indeksy, gdzie konwersja spowodowała 'NaT', co wskazuje na brakujące wartości
nan_indices = data_copy['Var13_converted'].isna()

# Porównujemy oryginalne wartości 'Var13', które po konwersji stały się 'NaT'
original_values_with_nan = data_copy.loc[nan_indices, 'Var13']

print("Oryginalne wartości kolumny 'Var13', które po konwersji stały się puste:")
print(original_values_with_nan)

# Próba przekonwertowania tych dat w elastyczniejszy sposób, aby uwzględnić możliwe anomalie
# Filtrujemy te, które nie są 'NaT' i sprawdzamy, czy są większe niż rok 2024
dates_after_2024 = original_values_with_nan.apply(
    lambda x: pd.to_datetime(x, format='%d%b%Y', errors='coerce')
).dropna() > pd.Timestamp('2024-12-31')

# Sprawdzamy, czy wszystkie prawidłowo przekonwertowane daty (poza 'NaT') są po roku 2024
wszystkie_daty_po_2024 = dates_after_2024.all()

print("Czy wszystkie daty 'Var13' po konwersji, które nie są 'NaT', są większe niż rok 2024?:", wszystkie_daty_po_2024)
# Czyli mamy 784 przypadki dla Var13 w ktorych daty to zapychacze 

Oryginalne wartości kolumny 'Var13', które po konwersji stały się puste:
5        31Dec9999
56       31Dec9999
67       31Dec9999
86       31Dec9999
95       31Dec9999
           ...    
49510    31Dec9999
49537    31Dec9999
49617    31Dec9999
49635    31Dec9999
49765    31Dec9999
Name: Var13, Length: 784, dtype: object
Czy wszystkie daty 'Var13' po konwersji, które nie są 'NaT', są większe niż rok 2024?: True


In [13]:
""" Teraz mozna stworzyc nowa kolumne np. WorkingYears=Var13-ApplicationDate, 
bedzie ona nam mowila ile pracownik pracuje na danym stanowisku od momentu zatrudnienia do wziecia kredytu
"""
# Tworzenie nowej kolumny working_months
sorted_data['working_months'] = ((sorted_data['application_date'] - sorted_data['Var13']) / np.timedelta64(1, 'M')).fillna(0).astype(int)

# Usuwanie kolumn 'application_date' i 'Var13'
sorted_data.drop(['application_date', 'Var13'], axis=1, inplace=True)

# Ustawienie 'working_months' jako pierwszej kolumny
# Pobieramy listę kolumn bez 'working_months'
columns_without_working_months = [col for col in sorted_data.columns if col != 'working_months']

# Tworzymy nową listę kolumn z 'working_months' na początku
new_columns_order = ['working_months'] + columns_without_working_months

# Reorganizacja DataFrame według nowej kolejności kolumn
sorted_data = sorted_data[new_columns_order]

print(sorted_data.head())




   working_months        ID  customer_id       _r_  Var9   Var10  Var1   Var4  \
0              62  11034977     32537148  0.267045  5640     NaN     1   7800   
1             160  11034978     32761663  0.533040  3000     NaN     1  11100   
2             248  11034979     32701063  0.861555  6240  4300.0     2   2400   
3             265  11034980     32386786  0.506451  4560  2500.0     3  11800   
4             248  11034981     32692110  0.210913  5160     NaN     1  10200   

   Var5  Var6  ...  Var3  Var11  Var12  Var14  target  Application_status  \
0    99     1  ...     1      5    NaN      2     0.0            Approved   
1    78     1  ...     2      2    NaN      4     0.0            Approved   
2    15     1  ...     1      5    5.0      1     0.0            Approved   
3    30     6  ...     2      6    5.0      1     0.0            Approved   
4    72     3  ...     1      5    NaN      0     NaN            Rejected   

   Var18  Var19  Var27  Var28  
0    1.0    NaN   

C:\Users\jansl\AppData\Local\Temp\ipykernel_18828\1164230245.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_data['working_months'] = ((sorted_data['application_date'] - sorted_data['Var13']) / np.timedelta64(1, 'M')).fillna(0).astype(int)
C:\Users\jansl\AppData\Local\Temp\ipykernel_18828\1164230245.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_data.drop(['application_date', 'Var13'], axis=1, inplace=True)


In [14]:
# Teraz mozemy zmienic zmienne w Application_status na zmienne binarne 

# Zamiana wartości 'Approved' na 1 i 'Rejected' na 0 w kolumnie 'Application_status'
sorted_data['Application_status'] = sorted_data['Application_status'].replace({'Approved': 1, 'Rejected': 0})

# Sprawdzenie zmian
print(sorted_data[['Application_status']].head())

   Application_status
0                   1
1                   1
2                   1
3                   1
4                   0


In [15]:
"""""
Teraz pora na one hot encoding dla tych zmiennych
ktore okreslamy jako kategoryczne nominalne 
W naszym wypadku bedzie to tak:
- Var2
- Var3 
- Var11
- Var12
- Var14
"""""
sorted_data_encoded = pd.get_dummies(sorted_data, columns=['Var2', 'Var3', 'Var11', 'Var12', 'Var14'])

print(sorted_data_encoded.head())


   working_months        ID  customer_id       _r_  Var9   Var10  Var1   Var4  \
0              62  11034977     32537148  0.267045  5640     NaN     1   7800   
1             160  11034978     32761663  0.533040  3000     NaN     1  11100   
2             248  11034979     32701063  0.861555  6240  4300.0     2   2400   
3             265  11034980     32386786  0.506451  4560  2500.0     3  11800   
4             248  11034981     32692110  0.210913  5160     NaN     1  10200   

   Var5  Var6  ...  Var12_3.0  Var12_4.0  Var12_5.0  Var12_6.0  Var12_7.0  \
0    99     1  ...          0          0          0          0          0   
1    78     1  ...          0          0          0          0          0   
2    15     1  ...          0          0          1          0          0   
3    30     6  ...          0          0          1          0          0   
4    72     3  ...          0          0          0          0          0   

   Var14_0  Var14_1  Var14_2  Var14_3  Var14_4  
0